### Intersection of mrcanavar, PacBio CCS, and ONT high coverage regions to identify potential CNV

Generating excessive coverage bed files using calculations from mosdepth for both PacBio CCS 15kb_20kb merged and ONT bam files.

HG004_GRCh37_ONT-UL_UCSC_20200508.bam is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/HG004_NA24143_mother/UCSC_Ultralong_OxfordNanopore_Promethion/HG004_GRCh37_ONT-UL_UCSC_20200508.bam

HG004.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x.trio.bam is from https://drive.google.com/file/d/13UyMXW8zm-6uxn9oKUZ9DaFS_dSmmRd6/view?usp=sharing

HG004.hs37d5.300x.bam.bilkentuniv.052119.dups.bed is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/BilkentUni_IlluminaHiSeq_TARDIS_mrCaNaVar_05212019/AJtrio-HG004.hs37d5.300x.bam.bilkentuniv.052119.dups.bed.gz

pbsv_HG004_SequelII.hs37d5.vcf.gz is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/PacBio_CCS_15kb_20kb_chemistry2_10312019/pbsv/pbsv_HG004_SequelII.hs37d5.vcf.gz

convert_mosdepth_to_excessive_coverage.py is at the end of this notebook

In [ ]:
### mosdepth commands

In [ ]:
mosdepth -b 1000 -x --mapq 20 --no-per-base HG004.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x_1000_window_size HG004.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x.trio.bam

samtools view -1 -F 0x100 HG004_GRCh37_ONT-UL_UCSC_20200508.bam -h > filtered_HG004_GRCh37_ONT-UL_UCSC_20200508.bam

mosdepth -b 1000 -x --no-per-base HG004_filtered_combined_GRCh37_1000_window_size filtered_HG004_GRCh37_ONT-UL_UCSC_20200508.bam

In [ ]:
### HG004_mrcanavar_intersect_ccs_window_size_cnv_threshold_intersect_ont_window_size_cnv_threshold.bed
### What this does: find potential CNVs in HG4 from intersecting coverage files from PacBio HiFi, ONT, and Illumina data. This generates PacBio HiFi excessive coverage bed, intersect with mrCaNaVar dups bed, generates ONT excessive coverage bed, intersects to these all to generate exclusion bed

In [ ]:
python convert_mosdepth_to_excessive_coverage.py --input HG004.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x_1000_window_size.regions.bed --output HG004.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x_1000_window_size_excessive_coverage_cnv_threshold.bed --threshold 77.7625

bedtools intersect -a AJtrio-HG004.hs37d5.300x.bam.bilkentuniv.052119.dups.bed -b HG004.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x_1000_window_size_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_ccs_cnv_threshold.bed

python convert_mosdepth_to_excessive_coverage.py --input HG004_filtered_combined_GRCh37_1000_window_size.regions.bed --output HG004_filtered_combined_GRCh37_1000_window_size_excessive_coverage_cnv_threshold.bed --threshold 106.3375

bedtools intersect -a mrcanavar_intersect_ccs_cnv_threshold.bed -b HG004_filtered_combined_GRCh37_1000_window_size_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_ccs_window_size_cnv_threshold_intersect_ont_window_size_cnv_threshold.bed


In [ ]:
### Find coverage levels of excessive coverage on CCS and ONT data in R

In [ ]:
chr_1_22 <- c("1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22")
mosdepth_HG004_CCS_15kb_20kb_merged_MAPQ20_GRCh37 = read.delim("HG004.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_HG004_CCS_15kb_20kb_merged_MAPQ20_GRCh37_chr_1_22 <- mosdepth_HG004_CCS_15kb_20kb_merged_MAPQ20_GRCh37[which(mosdepth_HG004_CCS_15kb_20kb_merged_MAPQ20_GRCh37[,"CHR"] %in% chr_1_22),] 
quantile(mosdepth_HG004_CCS_15kb_20kb_merged_MAPQ20_GRCh37_chr_1_22[,"DEPTH"])

IQR(mosdepth_HG004_CCS_15kb_20kb_merged_MAPQ20_GRCh37_chr_1_22[,"DEPTH"])
#: 17.83
(quantile(mosdepth_HG004_CCS_15kb_20kb_merged_MAPQ20_GRCh37_chr_1_22[,"DEPTH"])[3]/2)*2.5
#: 77.7625

unname(quantile(mosdepth_HG004_CCS_15kb_20kb_merged_MAPQ20_GRCh37_chr_1_22[,"DEPTH"])[4]) + unname(IQR(mosdepth_HG004_CCS_15kb_20kb_merged_MAPQ20_GRCh37_chr_1_22[,"DEPTH"]))*1.5
# 96.985

mosdepth_ONT_1000_window_size = read.delim("HG004_filtered_combined_GRCh37_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]

quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])

IQR(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])
#: 16.34
(quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])[3]/2)*2.5
#: 106.3375

In [ ]:
### GRCh37_union_HG004_CCS_15kb_20kb_merged_ONT_1000_window_size_combined_elliptical_outlier_threshold.bed

### What this does: Find another set of potential CNVs from computing a coverage threshold using an elliciptal outlier for PacBio HiFi and ONT CNV file steps in R

mosdepth_CCS_15kb_20kb_merged_1000_window_size = read.delim("HG004.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

mosdepth_ONT_1000_window_size = read.delim("HG004_filtered_combined_GRCh37_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

chr_1_22 <- c("1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22")

mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22 <- mosdepth_CCS_15kb_20kb_merged_1000_window_size[which(mosdepth_CCS_15kb_20kb_merged_1000_window_size[,"CHR"] %in% chr_1_22),]

mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size <- data.frame(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22) 

df_mosdepth_ONT_1000_window_size <- data.frame(mosdepth_ONT_1000_window_size_chr_1_22)

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined[,5] <- mosdepth_ONT_1000_window_size_chr_1_22[,4]

colnames(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined) <- c("CHR", "START", "END", "CCS_DEPTH", "ONT_DEPTH")

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined 

medianccsdepth = median(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22[,"DEPTH"]) 

medianontdepth = median(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"]) 

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6] <- sqrt(((df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined[,4]/medianccsdepth)^2 + (df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined[,5]/medianontdepth)^2)/2) 

threshold_ellipctial_outlier = unname(quantile(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6])[4]+(1.5*IQR(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6])[1]))

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_outliers <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[which(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6] > threshold_ellipctial_outlier),]

write.csv(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_outliers, file = "GRCh37_union_HG004_CCS_15kb_20kb_merged_ONT_1000_window_size_combined_elliptical_outlier_threshold.bed", row.names = FALSE)

In [ ]:
### pbsv_HG004_SequelII.GRCh37_gt49bp_slop50_repeatexpanded_slop25percent.bed

### What this does: find SV calls from PacBio HiFi that will be excluded from HG4 in addition to the v0.6 GIAB SV calls

In [ ]:
zgrep -v ^# pbsv_HG004_SequelII.hs37d5.vcf.gz | awk '{FS=OFS="\t"} {  if(length($4)>49 || length($5)>49) print $1,$2-50,$2+length($4)+50} '  > pbsv_HG004_SequelII.GRCh37.pbsv_gt49bp_slop50.bed

intersectBed -wa -a GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz -b pbsv_HG004_SequelII.GRCh37.pbsv_gt49bp_slop50.bed | multiIntersectBed -i stdin pbsv_HG004_SequelII.GRCh37.pbsv_gt49bp_slop50.bed | mergeBed -i stdin -d 1000 | awk '{FS=OFS="\t"} { print $1, $2-int(0.25*($3-$2)), $3+int(0.25*($3-$2))}' | awk '{FS=OFS="\t"} { if($2<0) $2=0; print}' > pbsv_HG004_SequelII.GRCh37.pbsv_gt49bp_slop50_repeatexpanded_slop25percent.bed

In [ ]:
#### convert_mosdepth_to_excessive_coverage.py script

import argparse

parser = argparse.ArgumentParser(description="Subset bed file to callable regions only")
parser.add_argument('--input_file', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output_file', metavar="O", type=str, nargs="+", help="output file")
parser.add_argument('--threshold', metavar="T", type=str, nargs="+", help="input threshold")
args = parser.parse_args()

f = open(args.input_file[0], "r") 
f_lines = f.readlines()

f_out = open(args.output_file[0], "w+")
threshold = float(args.threshold[0])

for line in f_lines:   
    if "DEPTH" in line: 
        continue
    line_split = line.split("\t")
    depth_field = float(line_split[3])
    if depth_field > threshold:
        f_out.write(line)
        f_out.flush()  

f.close()
f_out.close()